In [1]:
## Runs my implementation of Word2Vec
## The input is a text file plus some parameters 
## The pre-processing is performed by functions in TexttoDict

import numpy as np
import theano
import theano.tensor as T
from theano import shared
import nltk
import string
from collections import defaultdict
from TexttoDict import *

rng = np.random

In [2]:
#---------------------------------------
# C is the size of the bag of words
# N is the size of hidden layer
# V is length of dictcut
# eta is the learning rate

class NNet(object):
    def __init__(self,V,N,eta):

        # inititalizing the weight matrices    
        self.W1 = theano.shared(rng.randn(V,N), name="W1")   # a V x N matrix
        self.W2 = theano.shared(rng.randn(V,N), name="W2")   # a V x N matrix

# (x,y) are shared variables for (input, output)
# x is the input bag of words
# y is the target word, the middle word in the bag
# must be integers since they are indices for matrix elements
        x = T.ivector("x") # a vector, dtype=int32
        y = T.ivector("y") # a vector, dtype=int32

# This evaluates the hidden and output layers
# This is a linear operation, no logistic function unlike a typical neural net
# To be more precise we are evaluating these at the level of the hidden layer
        vhidden = self.W1[x] # a C x N matrix
        vout = self.W2[y] # a 1 x N matrix

        vhiddenvec= T.mean(vhidden,axis=0)  # an N-vector
        voutvec=vout.reshape((N,)) # an N-vector
    
# Entropy loss function (a scalar)
        Entropy = -T.dot(vhiddenvec,voutvec) + T.log(T.exp(T.dot(vhidden,self.W2.T)).sum())

# Compute the gradient of the cost wrt the weights
# This is the computational power of Theano
        gradW1, gradW2 = T.grad(Entropy, [self.W1, self.W2])

# The train function updates the weights and computes the entropy
        train = theano.function(
              inputs=[x,y],
              outputs=Entropy,
              updates=((self.W1, self.W1 - eta * gradW1),(self.W2, self.W2 - eta * gradW2)))
    
        self.train=train

#---------------------------------------
    # outarr is a vector of 
    def NNettrain(self,inarr, outarr):
        for i in range(len(inarr)):
            ent=self.train(inarr[i], [outarr[i]])
    
        return ent
    
    def returnweights(self):
        return self.W1


This provides the bags of words from the textfile
using functions from TexttoDict.py

In [3]:
W2Vdata=w2vdatafromtext("delorme.txt", contsize=7, trainlines=10000,freqcutoff=5)
inarr=np.array(W2Vdata[1]).astype(np.int32)
outarr=np.array(W2Vdata[2]).astype(np.int32)

In [4]:
V=len(W2Vdata[0])
N=100
eta=0.5

Initializes the net

In [5]:
W2VNet=NNet(V,N,eta)

Computes the final error

In [6]:
error = W2VNet.NNettrain(inarr, outarr)
print error

7.55842341489


returns the weights, this contains the vector representation of the words

In [ ]:
weights = W2VNet.returnweights()

In [ ]:
print weights.get_value()